In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

In [3]:
!kaggle competitions download -c fake-news

 93% 43.0M/46.5M [00:02<00:00, 25.4MB/s]
100% 46.5M/46.5M [00:02<00:00, 19.5MB/s]


In [58]:
import pandas as pd
import numpy as np


In [59]:
import csv
df= pd.read_csv('train.csv')

In [60]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [61]:
df.shape

(20800, 5)

In [62]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [63]:
df = df.dropna()

In [64]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [65]:
X = df.drop('label',axis=1)

In [66]:
y = df['label']

In [67]:
X.shape

(18285, 4)

In [68]:
y.shape

(18285,)

In [69]:
import tensorflow as tf


In [70]:
tf.__version__


'2.13.0'

In [71]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [73]:
voc_size=5000       # to take only 5000 words in vocabolary


In [74]:
messages = X.copy()

In [75]:
messages['title'][1]


'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [76]:
messages.reset_index(inplace=True)   #reset the index, as we have droped some rows above

In [77]:
import nltk
import re
from nltk.corpus import stopwords

In [78]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [79]:
# preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [80]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [81]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[684, 3207, 2967, 1368, 2493, 2766, 3760, 4044, 4352, 4984],
 [3354, 1720, 2980, 4951, 3747, 1470, 471],
 [3487, 2799, 2879, 969],
 [3636, 1464, 4242, 1935, 3742, 3789],
 [3265, 3747, 545, 4821, 1544, 2890, 3747, 1816, 1131, 4164],
 [1003,
  4236,
  124,
  1461,
  1221,
  3743,
  4405,
  3131,
  3807,
  1505,
  1886,
  1347,
  3420,
  2873,
  471],
 [1772, 2162, 1326, 774, 941, 1331, 4599, 1901, 1470, 2795, 290],
 [3593, 4528, 3901, 2569, 4027, 4678, 3743, 920, 1470, 2795, 290],
 [3910, 226, 477, 2459, 2287, 2931, 3003, 593, 3743, 4640],
 [4058, 1026, 2679, 235, 1536, 1266, 4236, 3368],
 [1247, 3279, 745, 434, 1109, 1632, 1351, 4794, 4677, 2099, 2435],
 [1935, 1732, 2493, 2931, 3743, 4027],
 [1422, 2787, 1465, 3425, 343, 2183, 1398, 4282, 765],
 [4078, 3617, 1047, 2317, 159, 1952, 2509, 1470, 2795, 290],
 [574, 3693, 4129, 878, 2015, 1470, 2795, 290],
 [944, 2148, 1653, 2160, 4100, 10, 3646, 2380, 2617, 320],
 [319, 300, 1720],
 [2114, 1068, 79, 1772, 3743, 2263, 1619, 471],
 [3220, 3

In [82]:
# inorder to make all the vectors of same, we will apply pad_sequencing

sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4044 4352 4984]
 [   0    0    0 ... 3747 1470  471]
 [   0    0    0 ... 2799 2879  969]
 ...
 [   0    0    0 ... 1470 2795  290]
 [   0    0    0 ... 3654 2086 1019]
 [   0    0    0 ... 1383 3544 4954]]


In [83]:
embedded_docs[0]


array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  684,
       3207, 2967, 1368, 2493, 2766, 3760, 4044, 4352, 4984], dtype=int32)

In [84]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [86]:
len(embedded_docs),y.shape   # checking both input and output has same length


(18285, (18285,))

In [88]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [89]:
X_final.shape,y_final.shape


((18285, 20), (18285,))

In [90]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [91]:
#training the model
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)


Epoch 1/10
192/192 [==============================] - 11s 41ms/step - loss: 0.3330 - accuracy: 0.8507 - val_loss: 0.1909 - val_accuracy: 0.9183
Epoch 2/10
192/192 [==============================] - 10s 53ms/step - loss: 0.1367 - accuracy: 0.9484 - val_loss: 0.2053 - val_accuracy: 0.9238
Epoch 3/10
192/192 [==============================] - 10s 52ms/step - loss: 0.0943 - accuracy: 0.9660 - val_loss: 0.2116 - val_accuracy: 0.9195
Epoch 4/10
192/192 [==============================] - 8s 40ms/step - loss: 0.0659 - accuracy: 0.9774 - val_loss: 0.2346 - val_accuracy: 0.9167
Epoch 5/10
192/192 [==============================] - 9s 47ms/step - loss: 0.0430 - accuracy: 0.9861 - val_loss: 0.2841 - val_accuracy: 0.9160
Epoch 6/10
192/192 [==============================] - 9s 46ms/step - loss: 0.0251 - accuracy: 0.9922 - val_loss: 0.4061 - val_accuracy: 0.9110
Epoch 7/10
192/192 [==============================] - 7s 39ms/step - loss: 0.0143 - accuracy: 0.9957 - val_loss: 0.4295 - val_accuracy: 0.9

In [93]:
# now checking performances
y_pred=model.predict(X_test)


189/189 [==============================] - 2s 7ms/step


In [94]:
from sklearn.metrics import confusion_matrix


In [102]:
y_pred[y_pred>0.5] = 1
y_pred[y_pred<=0.5] = 0


In [104]:
confusion_matrix(y_test,y_pred)


array([[3186,  233],
       [ 368, 2248]])

In [105]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9004142502071251

In [109]:
#using the dropout layer


from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [110]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)


Epoch 1/10
192/192 [==============================] - 12s 43ms/step - loss: 0.3472 - accuracy: 0.8353 - val_loss: 0.1937 - val_accuracy: 0.9196
Epoch 2/10
192/192 [==============================] - 9s 48ms/step - loss: 0.1486 - accuracy: 0.9439 - val_loss: 0.1876 - val_accuracy: 0.9216
Epoch 3/10
192/192 [==============================] - 10s 53ms/step - loss: 0.1061 - accuracy: 0.9611 - val_loss: 0.2026 - val_accuracy: 0.9216
Epoch 4/10
192/192 [==============================] - 13s 67ms/step - loss: 0.0854 - accuracy: 0.9703 - val_loss: 0.2322 - val_accuracy: 0.9208
Epoch 5/10
192/192 [==============================] - 7s 39ms/step - loss: 0.0664 - accuracy: 0.9749 - val_loss: 0.2752 - val_accuracy: 0.9193
Epoch 6/10
192/192 [==============================] - 9s 48ms/step - loss: 0.0504 - accuracy: 0.9836 - val_loss: 0.2941 - val_accuracy: 0.9137
Epoch 7/10
192/192 [==============================] - 11s 55ms/step - loss: 0.0382 - accuracy: 0.9868 - val_loss: 0.3173 - val_accuracy: 0.

In [113]:
y_pred=model.predict(X_test)
y_pred[y_pred>=0.5] = 1
y_pred[y_pred<0.5] = 0

189/189 [==============================] - 2s 12ms/step


In [114]:
accuracy_score(y_test,y_pred)

0.9153272576636289